# **파이토치 기초**

## 파이토치 패키지의 기본 구성

### **torch**
---
메인 네임스페이스. 텐서 등의 다양한 수학 함수가 포함되어 있으며 Numpy와 유사한 구조를 가진다.

### **torch.autograd**
---
자동 미분을 위한 함수들의 포함되어 있다. 

자동 미분의 on/off를 제어하는 콘텍스트 매니저나(enable_grad/no_grad)나 자체 미분 가능 함수를 정의할 때 사용하는 기반 클래스 'Function'등이 포함되어 있다.

### **torch.nn**
---
신경망을 구축하기 위한 다양한 데이터 구조나 레이어 등이 정의되어 있다. RNN, LSTM 같은 **레이어**, ReLu와 같은 **활성화 함수**, MSELoss와 같은 **손실 함수**들이 있다.


### **torch.optim**
---
확률적 경사 하강법(Stochastic Gradient Descent, SGD)를 중심으로 한 파라미터 최적화 알고리즘이 구현되어 있다.

### torch.utils.data
---
SGD의 반복 연산을 실행할 대 사용하는 미니 배치용 유틸리티 함수가 포함되어 있다.

### torch.onnx
---
ONNX(Open Neural Network Exchange)의 포맷으로 모델을 **익스포트(export)**할 때 사용한다. ONNX는 서로 다른 딥 러닝 프레임워크 간에 **모델을 공유**할 때 사용하는 포맷.

## 텐서 조작하기

In [2]:
import torch

### 1D

In [3]:
t = torch.FloatTensor([0., 1., 2., 3., 4., 5., 6.])
print(t)

tensor([0., 1., 2., 3., 4., 5., 6.])


In [4]:
print(t.dim())
print(t.shape)
print(t.size())

1
torch.Size([7])
torch.Size([7])


In [5]:
print(t[0], t[1], t[-1])
print(t[2:5], t[4:-1])
print(t[:2], t[3:])

tensor(0.) tensor(1.) tensor(6.)
tensor([2., 3., 4.]) tensor([4., 5.])
tensor([0., 1.]) tensor([3., 4., 5., 6.])


### 2D

In [6]:
t = torch.FloatTensor([[1., 2., 3.],
                       [4., 5., 6.],
                       [7., 8., 9.],
                       [10., 11., 12.]
                       ])
print(t)

tensor([[ 1.,  2.,  3.],
        [ 4.,  5.,  6.],
        [ 7.,  8.,  9.],
        [10., 11., 12.]])


In [7]:
print(t.dim())
print(t.size())

2
torch.Size([4, 3])


In [8]:
print(t[:, 1])
print(t[:, 1].size())

tensor([ 2.,  5.,  8., 11.])
torch.Size([4])


In [9]:
print(t[:, :-1])

tensor([[ 1.,  2.],
        [ 4.,  5.],
        [ 7.,  8.],
        [10., 11.]])


In [10]:
print(t[:-1, :])

tensor([[1., 2., 3.],
        [4., 5., 6.],
        [7., 8., 9.]])


In [11]:
print(t[:-1, :-1])

tensor([[1., 2.],
        [4., 5.],
        [7., 8.]])


### 브로드캐스팅(Broadcasting)

두 행렬 A, B의 덧셈과 뺄셈을 위해선 **행렬 A,B의 크기가 같아야** 한다는 조건이 있다. 

그리고 두 행렬이 곱셈을 할 때에는 **A의 열과 B의 행 차원**이 일치해야 한다는 조건도 있다.

하지만 딥러닝을 하다보면 불가피하게 크기가 다른 행렬(혹은 텐서)에 대해 사칙 연산을 수행해야 하는 경우가 생긴다. 파이토치에서는 자동으로 크기를 맞춰서 연산을 수행하게 만드는 **브토드캐스팅** 기능을 제공한다.

In [12]:
m1 = torch.FloatTensor([[3, 3]])
m2 = torch.FloatTensor([[2, 2]])
print(m1 + m2)

tensor([[5., 5.]])


1행 2열의 같은 크기의 행렬이기 때문에 문제 없이 덧셈 연산이 가능하다.

In [13]:
m1 = torch.FloatTensor([[1, 2]])
m2 = torch.FloatTensor([3]) # [3] -> [3, 3]
print(m1 + m2)

tensor([[4., 5.]])


원칙적으로는 (1,2)와 (1,)의 크기가 달라 덧셈이 불가능하지만 파이토치는 m2롤 동일한 크기로 변경하여 연산을 수행한다. 

In [14]:
# 2 x 1 Vector + 1 x 2 Vector
m1 = torch.FloatTensor([[1, 2]])
m2 = torch.FloatTensor([[3], [4]])
print(m1 + m2)

tensor([[4., 5.],
        [5., 6.]])


[1, 2]
==> [[1, 2],
     [1, 2]]

[3]
[4]
==> [[3, 3],
     [4, 4]]

파이토치는 위처럼 자동으로 브로드캐스팅하여 연산을 수행한다.

이는 편리한 기능이지만 자동으로 실행되는 기능인 만큼 사용자 입장에서 굉장히 주의해서 사용해야 한다. 크기가 다른 텐서를 크기가 같다고 착각하여 덧셈을 하게 되면 나중에 원하지 않는 결과가 나와도 어디서 문제가 발생했는지 찾기 굉장히 어려울 수 있다.

### 자주 사용되는 기능

#### 행렬 곱셈과 원소별 곱셈(Matrix Multiplication Vs. Multiplication)
*   행렬곱셈(.matmul)
*   원소 별 곱셈(.mul) = element-wise 곱셈


In [15]:
# 행렬곱셈 .matmul
m1 = torch.FloatTensor([[1, 2], [3, 4]])
m2 = torch.FloatTensor([[1], [2]])
print('Shape of Matrix 1: ', m1.shape) # 2 x 2
print('Shape of Matrix 2: ', m2.shape) # 2 x 1
print(m1.matmul(m2)) # 2 x 1

Shape of Matrix 1:  torch.Size([2, 2])
Shape of Matrix 2:  torch.Size([2, 1])
tensor([[ 5.],
        [11.]])


In [16]:
# 원소별 곱셈
m1 = torch.FloatTensor([[1, 2], [3, 4]])
m2 = torch.FloatTensor([[1], [2]])
print('Shape of Matrix 1: ', m1.shape) # 2 x 2
print('Shape of Matrix 2: ', m2.shape) # 2 x 1
print(m1 * m2) # 2 x 2
print(m1.mul(m2))

Shape of Matrix 1:  torch.Size([2, 2])
Shape of Matrix 2:  torch.Size([2, 1])
tensor([[1., 2.],
        [6., 8.]])
tensor([[1., 2.],
        [6., 8.]])


[1]

[2]

==>
 [[1, 1],
  [2, 2]]

  로 브로드캐스팅되어 각 원소별로 곱셈이 수행된 것을 확인할 수 있다.

#### 평균(Mean)

In [17]:
t = torch.FloatTensor([1, 2])
print(t.mean())

tensor(1.5000)


In [18]:
t = torch.FloatTensor([[1, 2], [3, 4]])
print(t)

tensor([[1., 2.],
        [3., 4.]])


In [19]:
print(t.mean())

tensor(2.5000)


4개의 원소 평균이 출력됨을 확인할 수 있다.

In [20]:
# 첫 번째 차원(행) 평균
print(t.mean(dim=0))

tensor([2., 3.])


dim은 해당 차원을 제거한다는 의미가 된다. 따라서 행이 제게되고 열끼리 평균을 계산한다는 뜻.

In [21]:
print(t.mean(dim=1))

tensor([1.5000, 3.5000])


In [22]:
print(t.mean(dim=-1)) # -1은 마지막을 의미 -> 열을 제거

tensor([1.5000, 3.5000])


#### 덧셈(sum)

In [23]:
t = torch.FloatTensor([[1, 2], [3, 4]])
print(t)

tensor([[1., 2.],
        [3., 4.]])


In [24]:
print(t.sum()) # 단순히 원소 전체의 덧셈을 수행
print(t.sum(dim=0)) # 행을 제거
print(t.sum(dim=1)) # 열을 제거
print(t.sum(dim=-1)) # 열을 제거

tensor(10.)
tensor([4., 6.])
tensor([3., 7.])
tensor([3., 7.])


sum은 원소를 모두 더해준다

#### 최대(Max)와 아그맥스(ArgMax)

Max는 원소의 **최대값**을 리턴하고, **아그맥스(ArgMax)**는 최대값을 가진 **인덱스**를 리턴한다.

In [25]:
t = torch.FloatTensor([[1, 2], [3, 4]])
print(t)

tensor([[1., 2.],
        [3., 4.]])


In [26]:
print(t.max())

tensor(4.)


In [27]:
print(t.max(dim=0)) # 열에서 최대

torch.return_types.max(
values=tensor([3., 4.]),
indices=tensor([1, 1]))


[1, 1]은 argmax로 인덱스를 의미한다. 첫 번째 열에서 3의 인덱스 1, 두 번째 열에서 4의 인덱스 1이 리턴된 것.

In [28]:
print('Max: ', t.max(dim=0)[0])
print('Argmax: ', t.max(dim=0)[1])

Max:  tensor([3., 4.])
Argmax:  tensor([1, 1])


만약 max따로 argmax따로 리턴받고 싶다면 리턴값에 인덱스를 부여하면 된다.

In [29]:
print(t.max(dim=1)) # 행별 최대값 출력
print(t.max(dim=-1))

torch.return_types.max(
values=tensor([2., 4.]),
indices=tensor([1, 1]))
torch.return_types.max(
values=tensor([2., 4.]),
indices=tensor([1, 1]))


#### 뷰(View) - 원소의 수를 유지하면서 텐서의 크기 변경하기(매우 중요)

In [30]:
import numpy as np

t = np.array([[[0, 1, 2],
               [3, 4, 5]],
              [[6, 7, 8],
               [9, 10, 11]]])
ft = torch.FloatTensor(t)

In [31]:
print(ft.shape)

torch.Size([2, 2, 3])


In [32]:
# 3차원 텐서에서 2차원 텐서로 변경
print(ft.view([-1,3])) # -1은 행이 몇 개가 되는 3열짜리 텐서로 만들라는 의미.
print(ft.view([-1,3]).shape)

tensor([[ 0.,  1.,  2.],
        [ 3.,  4.,  5.],
        [ 6.,  7.,  8.],
        [ 9., 10., 11.]])
torch.Size([4, 3])


In [33]:
# 3차원 텐서의 크기 변경
print(ft.view([-1, 1, 3]))
print(ft.view([-1, 1, 3]).shape)

tensor([[[ 0.,  1.,  2.]],

        [[ 3.,  4.,  5.]],

        [[ 6.,  7.,  8.]],

        [[ 9., 10., 11.]]])
torch.Size([4, 1, 3])


#### 스퀴즈(Squeeze) - 크기가 1인 차원을 제거

차원이 1인 경우 해당 차원 제거.

In [34]:
ft = torch.FloatTensor([[0], [1], [2]])
print(ft)
print(ft.shape)

tensor([[0.],
        [1.],
        [2.]])
torch.Size([3, 1])


In [35]:
print(ft.squeeze())
print(ft.squeeze().shape)

tensor([0., 1., 2.])
torch.Size([3])


squeeze는 손으로 무언가를 짜내는 것을 뜻한다. 즉, 불필요하게 차원이 존재하는 경우 차원을 제거하여 적은 차원으로 압축하는 것이라 생각할 수 있다.

#### 언스퀴즈(Unsqueeze) - 크기가 1인 차원을 추가


In [36]:
ft = torch.Tensor([0, 1, 2])
print(ft.shape)

torch.Size([3])


In [37]:
print(ft.unsqueeze(0)) # 0은 차원을 추가할 위치를 의미 
print(ft.unsqueeze(0).shape)

tensor([[0., 1., 2.]])
torch.Size([1, 3])


In [38]:
print(ft.unsqueeze(1))
print(ft.unsqueeze(1).shape)

tensor([[0.],
        [1.],
        [2.]])
torch.Size([3, 1])


In [39]:
print(ft.unsqueeze(-1)) # -1은 마지막 차원을 의미
print(ft.unsqueeze(-1).shape)

tensor([[0.],
        [1.],
        [2.]])
torch.Size([3, 1])


지금까지 살펴본 view(), squeeze(), unsqueeze()는 텐서의 **원소 수를 그대로 유지**하면서 모양과 차원을 바꾼다.

#### 타입 캐스팅(Type Casting)

텐서에는 자료형이 있다. 각 데이터형(Float, Long,..)별로 정의되어 있는데, 이 자료형을 변환하는 것을 **타입 캐스팅**이라고 한다.

In [40]:
lt = torch.LongTensor([1,2,3,4])
print(lt)

tensor([1, 2, 3, 4])


In [41]:
print(lt.float()) # float 형으로 타입 변경

tensor([1., 2., 3., 4.])


In [42]:
bt = torch.ByteTensor([True, False, False, True])
print(bt)

tensor([1, 0, 0, 1], dtype=torch.uint8)


In [43]:
print(bt.long())
print(bt.float())

tensor([1, 0, 0, 1])
tensor([1., 0., 0., 1.])


#### 연결하기(concatenate)


In [44]:
x = torch.FloatTensor([[1,2], [3,4]])
y = torch.FloatTensor([[5,6], [7,8]])

In [45]:
# cat으로 연결
print(torch.cat([x,y], dim=0)) # 0번째 차원을 늘려라

tensor([[1., 2.],
        [3., 4.],
        [5., 6.],
        [7., 8.]])


In [46]:
print(torch.cat([x,y], dim=1))

tensor([[1., 2., 5., 6.],
        [3., 4., 7., 8.]])


#### 스택킹(Stacking)

In [47]:
x = torch.FloatTensor([1,4])
y = torch.FloatTensor([2,5])
z = torch.FloatTensor([3,6])

In [48]:
print(torch.stack([x,y,z]))

tensor([[1., 4.],
        [2., 5.],
        [3., 6.]])


In [49]:
print(torch.stack([x,y,z], dim=1))

tensor([[1., 2., 3.],
        [4., 5., 6.]])


#### 0으로 채워진 텐서와 1로 채워진 텐서

In [50]:
x = torch.FloatTensor([[0, 1, 2], [2, 1, 0]])
print(x)

tensor([[0., 1., 2.],
        [2., 1., 0.]])


In [51]:
print(torch.ones_like(x))

tensor([[1., 1., 1.],
        [1., 1., 1.]])


In [52]:
print(torch.zeros_like(x))

tensor([[0., 0., 0.],
        [0., 0., 0.]])


#### in-place Operation(덮어쓰기 연산)

In [53]:
x = torch.FloatTensor([[1, 2], [3, 4]])

In [54]:
print(x.mul(2.))
print(x)

tensor([[2., 4.],
        [6., 8.]])
tensor([[1., 2.],
        [3., 4.]])


x 값은 변하지 않았다. 하지만 연산 뒤에 _를 붙이면 기존의 값을 덮어쓴다.

In [55]:
print(x.mul_(2.))
print(x)

tensor([[2., 4.],
        [6., 8.]])
tensor([[2., 4.],
        [6., 8.]])
